In [3]:
%pip install zenml
%pip install mlflow


  Using cached zenml-0.80.2-py3-none-any.whl.metadata (24 kB)
  Using cached pydantic_settings-2.8.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached sqlmodel-0.0.18-py3-none-any.whl.metadata (9.8 kB)
  Using cached ipywidgets-8.1.6-py3-none-any.whl.metadata (2.4 kB)
Using cached zenml-0.80.2-py3-none-any.whl (4.0 MB)
Using cached sqlmodel-0.0.18-py3-none-any.whl (26 kB)
Using cached pydantic_settings-2.8.1-py3-none-any.whl (30 kB)
Using cached ipywidgets-8.1.6-py3-none-any.whl (139 kB)


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\PRAVEEN ANANTH\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\ipywidgets\\widgets\\widget_media.py'
Consider using the `--user` option or check the permissions.



Note: you may need to restart the kernel to use updated packages.


In [5]:
from zenml import pipeline, step
import mlflow
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [7]:
class Evaluation:
    def calculate_scores(self, y_true, y_pred):
        return mean_squared_error(y_true, y_pred), r2_score(y_true, y_pred)

@step
def load_data():
    data = load_diabetes()
    X = pd.DataFrame(data.data, columns=data.feature_names)
    y = data.target
    return train_test_split(X, y, test_size=0.2, random_state=42)

@step
def train_model(X_train, X_test, y_train, y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return model, y_test, y_pred

@step
def evaluate_model(y_test, y_pred):
    evaluator = Evaluation()
    mse, r2 = evaluator.calculate_scores(y_test, y_pred)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2", r2)
    return r2

@step
def deployment_trigger(r2: float) -> bool:
    return r2 >= 0.5

@step
def deploy_model(model: LinearRegression, deploy: bool):
    if deploy:
        mlflow.sklearn.log_model(model, "model")

@pipeline
def continuous_deployment_pipeline():
    X_train, X_test, y_train, y_test = load_data()
    model, y_test, y_pred = train_model(X_train, X_test, y_train, y_test)
    r2 = evaluate_model(y_test, y_pred)
    deploy = deployment_trigger(r2)
    deploy_model(model, deploy)
